## MVP Engenharia de Dados - Notebook 1: Camada BRONZE

**Objetivo:** Realizar a ingestão (extração e carga) dos dados brutos para o Data Lake.

**Fontes de Dados (Ingestão):**

1.  **Dados Transacionais (Vendas):**
    * **Origem:** Arquivo `vendas_pdv.xlsx`.
    * **Conteúdo:** Registro detalhado de cada item vendido no restaurante (Notas Fiscais).
    * **Tabela de Destino:** `default.bronze_vendas_raw`.

2.  **Dados Cadastrais (Auxiliar de Grupos):**
    * **Origem:** Arquivo `grupos_produtos.xlsx`.
    * **Conteúdo:** Tabela de referência que vincula cada produto ao seu Grupo (ex: "1.01 AGUAS", "2.01 CERVEJAS").
    * **Propósito:** Essencial para categorizar os produtos e permitir a distinção entre comidas, bebidas... na análise final.
    * **Tabela de Destino:** `default.bronze_materiais_raw`.

In [0]:
# Download e instalação das bibliotecas do Pandas
%pip install openpyxl

In [0]:
import pandas as pd

# 1. Caminho do arquivo no volume
file_path = "/Volumes/workspace/default/dados_mvp_restaurante/vendas_pdv.xlsx"

# 2. Ler o arquivo com Pandas
try:
    pandas_df = pd.read_excel(file_path)
    
    # 3. Converter o DataFrame do Pandas para um DataFrame do Spark
    spark_df = spark.createDataFrame(pandas_df)
    
    # 4. Salvar como primeira tabela Delta (Bronze)
    table_name = "default.bronze_vendas_raw"
    spark_df.write.format("delta").mode("overwrite").saveAsTable(table_name)
    
    print(f"Sucesso! Tabela '{table_name}' criada.")
    
    # 5. Visualizar os dados
    display(spark_df)

except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")
    print("Verifique se o 'file_path' está 100% correto.")

In [0]:
# Verificando a contagem da Tabela de Vendas (Bronze)
# Lemos direto do catálogo para garantir que estamos contando o que foi salvo
vendas_salvas = spark.read.table("default.bronze_vendas_raw")
total_linhas = vendas_salvas.count()

print(f"O número total de linhas na tabela 'default.bronze_vendas_raw' é: {total_linhas}")

In [0]:
import pandas as pd

# 1. Caminho do arquivo no volume
caminho_arquivo_materiais = "/Volumes/workspace/default/dados_mvp_restaurante/grupos_produtos.xlsx" 

# 2. Ler arquivo com Pandas
try:
   
    pdf_materiais = pd.read_excel(caminho_arquivo_materiais, engine='openpyxl')
    
    # 3. Converter o DataFrame do Pandas para um DataFrame do Spark
    df_materiais_raw = spark.createDataFrame(pdf_materiais)

    # 3. Salvar na tabela Bronze
    df_materiais_raw.write.format("delta").mode("overwrite").saveAsTable("default.bronze_materiais_raw")

    print("Sucesso! Tabela 'default.bronze_materiais_raw' criada.")
    display(df_materiais_raw.limit(5))

except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")
    print("Verifique se o 'file_path' está 100% correto.")